In [1]:
import gdax
import pandas as pd
import time
import os.path
import requests
from shutil import copyfile
import cProfile
import datetime
import talib
import numpy as np

n = 4
public_client = gdax.PublicClient()

buy = False
sell = False
bidding = False
asking = False

PLN = 0 
BTC = 1
PLN_min = 1e-1
BTC_min = 1e-6

cci_thr = -60
mfi_thr = 0
cmo_thr = -120

# API_p = '557c4731915037e278adcb3aa96c029a'
# API_s = 'nt/cMZQ5rvxlDrK8qhw2wtqqZD16q3VyINlTpG6+Ba8pB5SsMWpS7VcyROPqZrFjAbD/dwEi0ucNsSCWGBzOsg=='

class Strategy:
    def __init__(self, param):
        self.time = datetime.datetime.now()
        self.c1 = param['c1']
        self.c2 = param['c2'] 
        self.c2_min = 0.001
        self.n = param['n']
        self.mfi_thr = param['mfi_thr']
        self.cci_thr = param['cci_thr']
        self.cmo_thr = param['cmo_thr']
        self.buy = False
        self.sell = False
        self.bidding = False
        self.asking = False
        self.queue = 0
        self.bid = 0
        self.ask = 0
        self.name = str(self.n) + '_' + str(self.mfi_thr) + '_' + str(self.cci_thr) + '_' + str(self.cmo_thr) + '_'

    def update(self, time, trades_ohlc, orderbook_l1):
        self.time = time
        average = trades_ohlc['close'].iloc[-1]
        equilibrium = self.c1/(self.c1+self.c2*average)
        
        window = trades_ohlc.iloc[-(self.n+1):]
    
        mfi = talib.MFI(window['high'].values, window['low'].values, window['close'].values, window['size'].values, n)[-1]
        cci = talib.CCI(window['high'].values, window['low'].values, window['close'].values, n)[-1]
        cmo = talib.CMO(window['close'].values, n)[-1]
        
        if cci > self.cci_thr and mfi > self.mfi_thr and cmo > self.cmo_thr:
            sell_new = True
        else:
            sell_new = False

        if cci < -self.cci_thr and mfi < 100 - self.mfi_thr and cmo < -self.cmo_thr:
            buy_new = True
        else:
            buy_new = False

        if(equilibrium > 0.01 and equilibrium < 0.99):
            if(self.buy and not buy_new and not sell_new):
                buy_new = True
            elif(self.sell and not sell_new and not buy_new):
                sell_new = True

        self.buy = buy_new
        self.sell = sell_new

        bids = orderbook_l1['bids'][0]
        asks = orderbook_l1['asks'][0]
        best_bid = float(bids[0])
        best_ask = float(asks[0])
        difference = round(best_ask - best_bid, 2)

        if self.bidding and self.bid != best_bid:
            if self.bid > best_bid + 0.01:
                self.c2 = self.c1/self.bid
                self.c1 = 0
#             print('remove bid')
            self.bidding = False
        elif self.bidding:
            if self.queue - float(bids[1]) > 0:
                self.queue -= float(bids[1])
            if self.queue < 0:
                self.c2 = self.c1/self.bid
                self.c1 = 0
#                 print('remove bid')
                self.bidding = False

        if(self.asking and self.ask != best_ask):
            if self.ask < best_ask - 0.01:
                self.c1 = self.c2*self.ask
                self.c2 = 0
#             print('remove ask')
            self.asking = False
        elif self.asking:
            if self.queue - float(asks[1]) > 0:
                self.queue -= float(asks[1])
            if self.queue < 0:
                self.c1 = self.c2*self.ask
                self.c2 = 0
#                 print('remove ask')
                self.asking = False

        if(self.buy and not self.bidding and self.c1 > self.c2_min*average):
            if(difference != 0.01):
#                 print('buy', best_bid + 0.01)    
                self.bid = best_bid + 0.01
                self.queue = self.c1/average
            else:
#                 print('buy', best_bid)
                self.bid = best_bid
                self.queue = float(bids[1])
            self.bidding = True


        if(self.sell and not self.asking and self.c2 > self.c2_min):
            if(difference != 0.01):
#                 print('sell', best_ask - 0.01)
                self.ask = best_ask - 0.01
                self.queue = self.c2
                queue_my = self.queue
            else:
#                 print('sell', best_ask)       
                self.ask = best_ask
                self.queue = float(asks[1])
            self.asking = True
            
        simulation = pd.DataFrame([[self.time, self.n, self.cci_thr, self.mfi_thr, self.cmo_thr, self.c1 + self.c2*average, self.c1/average + self.c2]], \
                            columns=['time', 'n', 'cci_thr', 'mfi_thr', 'cmo_thr', 'profit_c1', 'profit_c2'])
        simulation.to_csv('Simulation/GDAX_1650_02032018_300s.csv', mode='a', header=0)
        return self.c1/average + self.c2, self.name


In [2]:
strategies = []


ns = np.linspace(10, 20, 11, dtype=np.int16)  #4
mfis = np.linspace(0, 90, 10) # 0
ccis = np.linspace(0, 90, 10) # 95
cmos = np.linspace(0, 90, 10) # 10

param= {}
c1 = 0
c2 = 1

for n in ns:
    for mfi_thr in mfis:
        for cci_thr in ccis:
            for cmo_thr in cmos:
                param['c1'] = c1
                param['c2'] = c2
                param['n'] = n
                param['mfi_thr'] = mfi_thr
                param['cci_thr'] = cci_thr
                param['cmo_thr'] = cmo_thr
                strategies.append(Strategy(param))
                
print(len(strategies))
                


# for i in range(1):
while True:    
    for attempt in range(3):
        try:
            historic_ohlc = public_client.get_product_historic_rates('BTC-EUR', granularity=300)
            orderbook_l1 = public_client.get_product_order_book('BTC-EUR', level=1)
        except:
            time.sleep(0.25)
        else:
            break
    else:
        print('Proba polaczenia nieudana')

    t = datetime.datetime.now()
        
    trades_ohlc = pd.DataFrame(data=historic_ohlc, columns=['date','low','high','open','close','size'])
    trades_ohlc.index = trades_ohlc['date'].values
    trades_ohlc = trades_ohlc.sort_index()
    
    profit_max = 0
    for strategy in strategies:
        profit, name = strategy.update(t, trades_ohlc, orderbook_l1)
        if profit > profit_max:
            profit_max = profit
            name_max = name 
    print(profit_max, name_max)
        
    print(datetime.datetime.now() - t)
    print(orderbook_l1)
    print()


11000
1.0 10_0.0_0.0_0.0_
0:07:38.780557
{'sequence': 3430823411, 'bids': [['8817.01', '3.60986864', 10]], 'asks': [['8817.02', '0.26972138', 5]]}
0 1

1.0 10_0.0_0.0_0.0_
0:01:16.688982
{'sequence': 3430842530, 'bids': [['8820.01', '3.78239132', 9]], 'asks': [['8820.02', '0.8198522', 4]]}
0 1

1.0 10_0.0_0.0_0.0_
0:01:17.583762
{'sequence': 3430843826, 'bids': [['8820.01', '3.75163651', 11]], 'asks': [['8820.02', '0.4298522', 2]]}
0 1

1.0 10_0.0_0.0_0.0_
0:01:16.501434
{'sequence': 3430845020, 'bids': [['8820.01', '2.11677426', 12]], 'asks': [['8820.02', '0.0148522', 3]]}
0 1

1.0 10_0.0_0.0_0.0_
0:01:16.400288
{'sequence': 3430854913, 'bids': [['8825.72', '0.47238099', 8]], 'asks': [['8827.9', '0.13394819', 2]]}
0 1

1.0 10_0.0_0.0_0.0_
0:01:17.184195
{'sequence': 3430860063, 'bids': [['8827.89', '0.03099747', 3]], 'asks': [['8827.9', '0.05564574', 2]]}
0 1

1.0 10_0.0_0.0_0.0_
0:01:17.474217
{'sequence': 3430862004, 'bids': [['8827.99', '0.3569281', 10]], 'asks': [['8828', '1.01112

1.00278462957 20_0.0_30.0_10.0_
0:01:16.766529
{'sequence': 3430991770, 'bids': [['8809.09', '1.39255901', 6]], 'asks': [['8809.1', '8.27233957', 19]]}
0 1

1.00278462957 20_0.0_30.0_10.0_
0:01:17.143678
{'sequence': 3430993716, 'bids': [['8809.09', '1.56864979', 7]], 'asks': [['8809.1', '8.02433957', 19]]}
0 1

1.00278462957 20_0.0_30.0_10.0_
0:01:17.161568
{'sequence': 3430996208, 'bids': [['8809.09', '0.84020523', 5]], 'asks': [['8809.1', '4.65367108', 15]]}
0 1

1.00278462957 20_0.0_30.0_10.0_
0:01:17.656069
{'sequence': 3430999224, 'bids': [['8809.99', '0.41045059', 3]], 'asks': [['8810', '2.10103757', 4]]}
0 1

1.00278462957 20_0.0_30.0_10.0_
0:01:17.511113
{'sequence': 3431001413, 'bids': [['8809.99', '2.6283', 9]], 'asks': [['8810', '1.88114985', 6]]}
0 1

1.00278462957 20_0.0_30.0_10.0_
0:01:17.155684
{'sequence': 3431003738, 'bids': [['8814.3', '0.084665', 1]], 'asks': [['8814.99', '0.01', 1]]}
0 1

1.00278462957 20_0.0_30.0_10.0_
0:01:16.866238
{'sequence': 3431007910, 'bids

1.00515070373 20_0.0_70.0_10.0_
0:01:16.951618
{'sequence': 3431198993, 'bids': [['8845.07', '9.83839543', 15]], 'asks': [['8845.08', '1.42133278', 7]]}
0 1

1.00515070373 20_0.0_70.0_10.0_
0:01:17.062693
{'sequence': 3431201621, 'bids': [['8845.07', '9.66146439', 13]], 'asks': [['8845.08', '3.73494476', 3]]}
0 1

1.00515070373 20_0.0_70.0_10.0_
0:01:17.584610
{'sequence': 3431204876, 'bids': [['8845.07', '7.73146439', 11]], 'asks': [['8845.08', '4.53726432', 4]]}
0 1

1.00515184013 20_0.0_70.0_10.0_
0:01:17.477553
{'sequence': 3431208573, 'bids': [['8845.07', '7.99062883', 10]], 'asks': [['8845.08', '3.43000882', 3]]}
0 1

1.00515070373 20_0.0_70.0_10.0_
0:01:18.128779
{'sequence': 3431212451, 'bids': [['8845.07', '8.39195602', 14]], 'asks': [['8845.08', '5.83222344', 4]]}
0 1

1.00515070373 20_0.0_70.0_10.0_
0:01:18.617664
{'sequence': 3431216687, 'bids': [['8845.07', '8.52443203', 12]], 'asks': [['8845.08', '5.96557351', 6]]}
0 1

1.00515070373 20_0.0_70.0_10.0_
0:01:17.159204
{'seq

1.00486782094 20_0.0_70.0_10.0_
0:01:18.031522
{'sequence': 3431422546, 'bids': [['8847.57', '7.56529712', 23]], 'asks': [['8847.58', '5.65943092', 6]]}
0 1

1.00486782094 20_0.0_70.0_10.0_
0:01:18.254115
{'sequence': 3431428489, 'bids': [['8847.57', '6.80913712', 24]], 'asks': [['8847.58', '5.29913654', 2]]}
0 1

1.00486668519 20_0.0_70.0_10.0_
0:01:18.247095
{'sequence': 3431439945, 'bids': [['8842.25', '0.59132', 2]], 'asks': [['8844.16', '0.07935269', 1]]}
0 1

1.0048757713 20_0.0_70.0_10.0_
0:01:18.544887
{'sequence': 3431457598, 'bids': [['8842.63', '0.299', 1]], 'asks': [['8842.75', '0.001', 1]]}
0 1

1.00491552495 20_0.0_70.0_10.0_
0:01:18.824630
{'sequence': 3431465952, 'bids': [['8844.68', '0.05', 2]], 'asks': [['8847.5', '2.8354', 1]]}
0 1

1.00503594091 20_0.0_70.0_10.0_
0:01:19.475360
{'sequence': 3431471277, 'bids': [['8847.57', '0.59132', 3]], 'asks': [['8847.58', '0.72959471', 1]]}
0 1

1.00489848728 20_0.0_70.0_10.0_
0:01:17.910606
{'sequence': 3431480362, 'bids': [['8

1.0 10_0.0_0.0_0.0_
0:01:18.010465
{'sequence': 3431667969, 'bids': [['8924.99', '3.35143388', 9]], 'asks': [['8925', '10.70497923', 20]]}
0 1

1.0 10_0.0_0.0_0.0_
0:01:17.663543
{'sequence': 3431671580, 'bids': [['8924.99', '3.35382222', 9]], 'asks': [['8925', '8.81964846', 22]]}
0 1

1.0 10_0.0_0.0_0.0_
0:01:17.676160
{'sequence': 3431674428, 'bids': [['8924.99', '3.21020497', 12]], 'asks': [['8925', '6.83691402', 20]]}
0 1

1.0 10_0.0_0.0_0.0_
0:01:18.338118
{'sequence': 3431678506, 'bids': [['8929.99', '3.18576661', 13]], 'asks': [['8930', '0.07156516', 2]]}
0 1

1.0 10_0.0_0.0_0.0_
0:01:17.369712
{'sequence': 3431682850, 'bids': [['8960.98', '3.67900556', 9]], 'asks': [['8960.99', '0.41390211', 2]]}
0 1

1.0 10_0.0_0.0_0.0_
0:01:19.088332
{'sequence': 3431687010, 'bids': [['8967.72', '4.95351066', 15]], 'asks': [['8967.73', '0.10310332', 1]]}
0 1

1.0 10_0.0_0.0_0.0_
0:01:20.166585
{'sequence': 3431691057, 'bids': [['8970.59', '3.72841527', 12]], 'asks': [['8970.6', '0.78319872', 

...

1.0 10_0.0_0.0_0.0_
0:00:58.797999
{'sequence': 3433309865, 'bids': [['9168.28', '0.29244404', 5]], 'asks': [['9168.29', '0.02648194', 1]]}
0 1

1.0 10_0.0_0.0_0.0_
0:00:54.670124
{'sequence': 3433313935, 'bids': [['9169.99', '1.75951551', 7]], 'asks': [['9170', '0.60617336', 3]]}
0 1

1.0 10_0.0_0.0_0.0_
0:00:55.971474
{'sequence': 3433316853, 'bids': [['9169.99', '2.42581847', 9]], 'asks': [['9170', '0.5744035', 4]]}
0 1

1.0 10_0.0_0.0_0.0_
0:00:56.139308
{'sequence': 3433319459, 'bids': [['9170.06', '0.579', 2]], 'asks': [['9170.91', '1.213', 2]]}
0 1

1.0 10_0.0_0.0_0.0_
0:00:59.125057
{'sequence': 3433323767, 'bids': [['9170.22', '0.1029209', 2]], 'asks': [['9171', '0.0019', 1]]}
0 1

1.0 10_0.0_0.0_0.0_
0:00:56.264387
{'sequence': 3433326416, 'bids': [['9170.72', '0.6', 2]], 'asks': [['9171.49', '0.04', 1]]}
0 1

1.0 10_0.0_0.0_0.0_
0:00:58.649752
{'sequence': 3433328395, 'bids': [['9172.09', '0.41256', 2]], 'asks': [['9172.1', '0.07251611', 1]]}
0 1

1.0 10_0.0_0.0_0.0_
0:00:56

1.0 10_0.0_0.0_0.0_
0:00:59.726741
{'sequence': 3433492091, 'bids': [['9154.98', '2.08350796', 7]], 'asks': [['9154.99', '2.73178184', 5]]}
0 1

1.0 10_0.0_0.0_0.0_
0:00:56.480318
{'sequence': 3433494272, 'bids': [['9154.98', '3.18335895', 12]], 'asks': [['9154.99', '2.37574375', 6]]}
0 1

1.0 10_0.0_0.0_0.0_
0:00:59.325113
{'sequence': 3433496978, 'bids': [['9157.98', '2.37606413', 12]], 'asks': [['9157.99', '0.07345583', 2]]}
0 1

1.0 10_0.0_0.0_0.0_
0:00:56.147990
{'sequence': 3433501047, 'bids': [['9166.08', '0.43514351', 7]], 'asks': [['9166.09', '1.40529527', 4]]}
0 1

1.0 10_0.0_0.0_0.0_
0:00:59.858288
{'sequence': 3433503968, 'bids': [['9166.08', '4.36024432', 6]], 'asks': [['9166.09', '0.46893927', 5]]}
0 1

1.0 10_0.0_0.0_0.0_
0:00:56.714371
{'sequence': 3433506720, 'bids': [['9166.08', '3.76201173', 5]], 'asks': [['9166.09', '0.20478153', 7]]}
0 1

1.0 10_0.0_0.0_0.0_
0:00:59.951752
{'sequence': 3433509464, 'bids': [['9168.22', '0.7', 3]], 'asks': [['9168.23', '0.04803975', 

1.0 10_0.0_0.0_0.0_
0:00:59.294118
{'sequence': 3433624182, 'bids': [['9166.1', '1.28321519', 4]], 'asks': [['9166.11', '8.2311784', 8]]}
0 1

1.0 10_0.0_0.0_0.0_
0:00:56.454661
{'sequence': 3433625246, 'bids': [['9166.1', '0.48638527', 3]], 'asks': [['9166.11', '8.19628618', 10]]}
0 1

1.0 10_0.0_0.0_0.0_
0:00:56.887902
{'sequence': 3433626638, 'bids': [['9166.1', '0.73672783', 6]], 'asks': [['9166.11', '7.0144971', 12]]}
0 1

1.0 10_0.0_0.0_0.0_
0:00:55.284064
{'sequence': 3433628006, 'bids': [['9166.1', '0.70760496', 6]], 'asks': [['9166.11', '9.69444774', 18]]}
0 1

1.0 10_0.0_0.0_0.0_
0:00:58.626685
{'sequence': 3433629483, 'bids': [['9166.1', '0.54354748', 5]], 'asks': [['9166.11', '8.79238264', 16]]}
0 1

1.0 10_0.0_0.0_0.0_
0:00:57.328811
{'sequence': 3433631008, 'bids': [['9166.09', '0.55360642', 3]], 'asks': [['9166.1', '6.58482498', 16]]}
0 1

1.0 10_0.0_0.0_0.0_
0:00:59.250754
{'sequence': 3433632242, 'bids': [['9166', '11.1628727', 5]], 'asks': [['9166.01', '4.13124935', 1

1.0 10_0.0_0.0_0.0_
0:00:57.750144
{'sequence': 3433734769, 'bids': [['9150', '0.84423834', 2]], 'asks': [['9150.01', '6.96329707', 10]]}
0 1

1.0 10_0.0_0.0_0.0_
0:01:00.014929
{'sequence': 3433737026, 'bids': [['9150', '0.62323834', 2]], 'asks': [['9150.01', '7.48364855', 13]]}
0 1



KeyboardInterrupt: 